In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [2]:
from ivtmetrics.ivtmetrics import Recognition as ivtmetricsRecognition
import numpy as np

## Get IVT metrics

In [3]:
recognitionObject = ivtmetricsRecognition(num_class=100)

In [4]:

from os.path import join
import os
import json
import numpy as np
from utils.general.read_files import read_from_json 

from utils.convert_to_coco.convert_dataset_to_instrument_class_coco import get_bbox_info_from_coco_contour_xy
from utils.general.dataset_variables import TripletSegmentationVariables


def get_triplets_for_classification_metric_calculation_in_img(json_dict: dict):
    
    TRIPLET_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['triplet']
    TRIPLET_CLASS_TO_ID_DICT = {triplet_class: triplet_id for triplet_id, triplet_class in TRIPLET_ID_TO_CLASS_DICT.items()}
    
    # Get the contours together in a class_id, group_id tuple
    refactored_json_dict = {}
    for contour_info in json_dict['shapes']:
        class_name = contour_info['label']
        instance_id =  contour_info['group_id']
        contour_info = {
            "contour_pts" : contour_info['points'], 
            "score": contour_info.get('score', 1.0),
            "verb": contour_info['verb'],
            "verb_score": contour_info.get('verb_score', 1.0),
            "target": contour_info['target'],
            "target_score": contour_info.get('target_score', 1.0)
        }
        
        
        if((class_name, instance_id) in refactored_json_dict): 
            refactored_json_dict[(class_name, instance_id)].append(contour_info)    
        else:
            refactored_json_dict[(class_name, instance_id)] =  [contour_info] 
        
  
    # Rearrange for calculation of classification ivt.      
    triplets_in_img_presence_ids = np.zeros([1,100], dtype=float)
    triplet_in_img_names = []
    for (class_name, instance_id), contour_info_for_class_and_instance_id_list in refactored_json_dict.items():
        
        verb = contour_info_for_class_and_instance_id_list[0]['verb']
        target = contour_info_for_class_and_instance_id_list[0]['target']
        
        triplet_name=f'{class_name},{verb},{target}'
        
        # Need to replace this code.         
        if triplet_name not in TRIPLET_CLASS_TO_ID_DICT.keys():
            triplet_name = f'{class_name},null_verb,null_target'
        
        triplet_id = int(TRIPLET_CLASS_TO_ID_DICT[triplet_name])-1 
        
        triplets_in_img_presence_ids[0, triplet_id] = 1.0     
        triplet_in_img_names.append(triplet_name)     
    
    return triplets_in_img_presence_ids, triplet_in_img_names


def run_ivt_metric_object_for_folder_classification(ivt_metric_object,
                                    pred_ann_dir, 
                                    gt_ann_dir):  

    ivt_metric_object.reset() 
    pred_ann_list = sorted(os.listdir(pred_ann_dir))

    for i, filename in enumerate(pred_ann_list):    
        pred_ann_path = join(pred_ann_dir, filename)
        gt_ann_path =  join(gt_ann_dir, filename)     
        print(f'currently on {i}, {filename}')  
        
        
        json_dict_pred =  read_from_json(pred_ann_path)
        triplets_in_img_presence_ids_pred,  triplet_in_img_names_pred = get_triplets_for_classification_metric_calculation_in_img(json_dict_pred) 
        
        json_dict_gt =  read_from_json(gt_ann_path)
        triplets_in_img_presence_ids_gt,  triplet_in_img_names_gt = get_triplets_for_classification_metric_calculation_in_img(json_dict_gt)  
        
        print(f'pred, {triplet_in_img_names_pred}')
        print(f'gt, {triplet_in_img_names_gt}')

        ivt_metric_object.update(triplets_in_img_presence_ids_gt, triplets_in_img_presence_ids_pred )
        

In [5]:
pred_ann_dir = f'../resnet_model/work_dirs/rendezvous_cholectriplet_seg/combine_first_and_second_stage_results'  
gt_ann_dir = f'../data/triplet_segmentation_dataset_v2_second_stage/test/ann_dir'


run_ivt_metric_object_for_folder_classification(ivt_metric_object=recognitionObject,
                                 pred_ann_dir=pred_ann_dir,
                                 gt_ann_dir=gt_ann_dir)


currently on 0, t50_VID14_000000.json
pred, []
gt, []
currently on 1, t50_VID14_000001.json
pred, []
gt, []
currently on 2, t50_VID14_000002.json
pred, []
gt, []
currently on 3, t50_VID14_000003.json
pred, []
gt, []
currently on 4, t50_VID14_000004.json
pred, []
gt, []
currently on 5, t50_VID14_000005.json
pred, []
gt, []
currently on 6, t50_VID14_000006.json
pred, []
gt, []
currently on 7, t50_VID14_000007.json
pred, []
gt, []
currently on 8, t50_VID14_000008.json
pred, []
gt, []
currently on 9, t50_VID14_000009.json
pred, []
gt, []
currently on 10, t50_VID14_000010.json
pred, []
gt, []
currently on 11, t50_VID14_000012.json
pred, ['grasper,grasp,gallbladder']
gt, ['grasper,retract,gallbladder']
currently on 12, t50_VID14_000013.json
pred, ['grasper,grasp,gallbladder']
gt, ['grasper,retract,gallbladder']
currently on 13, t50_VID14_000014.json
pred, ['grasper,retract,gallbladder']
gt, ['grasper,retract,gallbladder']
currently on 14, t50_VID14_000015.json
pred, []
gt, []
currently on 15

In [9]:
results_i = recognitionObject.compute_global_AP('i')

results_v = recognitionObject.compute_global_AP('v')

results_t = recognitionObject.compute_global_AP('t')

results_iv = recognitionObject.compute_global_AP('iv')

results_it = recognitionObject.compute_global_AP('it')

results_ivt = recognitionObject.compute_global_AP('ivt')


print(f'results_i, {results_i['mAP']*100}' )
print(f'results_v, {results_v['mAP']*100}' )
print(f'results_t, {results_t['mAP']*100}' )
print(f'results_iv, {results_iv['mAP']*100}' )
print(f'results_it, {results_it['mAP']*100}' )
print(f'results_ivt, {results_ivt['mAP']*100}' )

c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_se

results_i, 81.7713189730266
results_v, 33.47840609404457
results_t, 19.341929569504064
results_iv, 15.587221208199772
results_it, 10.222095906466215
results_ivt, 8.319966774830668


c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_se